<div style="float:right; color:red; font-weight:bold;">Rename this file before you work on it!</div>
# Spark DataFrames Exercise 1

We use the [Adventure Works dataset](https://msdn.microsoft.com/en-us/library/hh403424.aspxto) to compare queries
in SQL and Spark.

## Task
1. Fill in the Python-Spark expressions to produce a table as described in the problem statements.
2. The SQL expression may give you a hint. It also allows you to see both systems side-by-side.
3. Documentation at http://spark.apache.org/docs/latest/sql-programming-guide.html

## Setup

### Spark Environment
This example runs with a Jupyter kernel that launches a Spark Context by default. If this is not the case on your system you have to initialize the Spark environment first.

In [ ]:
#"""
#Load packages and create context objects...
#"""
import os
import platform
import sys
import pandas

if not 'sc' in vars():
    print """
***********************************************
*** Warning: Spark needs to be initialized ****
***********************************************
    """
else:
    print("""Already running
          ____              __
         / __/__  ___ _____/ /__
        _\ \/ _ \/ _ `/ __/  '_/
       /__ / .__/\_,_/_/ /_/\_\   version %s
          /_/
    """ % sc.version)

if not 'sqlCtx' in vars():
    sqlCtx = SQLContext(sc)
print 'Spark Context available as `sc`'
print 'Spark SQL Context (%s) available as `sqlCtx`'%str(type(sqlCtx))
### The spark context doesn't really know the URL to the job manager... this would be different on every system
print "Monitor this application at http://data-science-02.atl.primedia.com:8088/proxy/"+sc.applicationId



## Additional Spark Packages for Python

In [ ]:
import pyspark.sql.functions as F

## Load Tables

Spark can load from a wide variety of data sources and format. CSV is the worst choice. Though, sometimes that's what we get.
You may have noticed that we had to ship a special 'jar' file when creating the SparkContext. Here's why:

In [ ]:
%%time
Employees_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=False)\
    .load('/user/pmolnar/data/AdventureWorks/Employees.csv.gz')

In [ ]:
Employees_df.printSchema()

Notice, they're all strings.

**Don't do the following on a very large data file!**

In [ ]:
%%time
Employees_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=True)\
    .load('/user/pmolnar/data/AdventureWorks/Employees.csv.gz')

In [ ]:
Employees_df.printSchema()

<pre>-rw-r--r--   3 pmolnar hadoop        466 2017-02-07 22:46 /user/pmolnar/data/AdventureWorks/Customer.csv.gz
-rw-r--r--   3 pmolnar hadoop      18125 2017-02-07 22:46 /user/pmolnar/data/AdventureWorks/Employees.csv.gz
-rw-r--r--   3 pmolnar hadoop        603 2017-02-07 22:46 /user/pmolnar/data/AdventureWorks/ItemsOrdered.csv.gz
-rw-r--r--   3 pmolnar hadoop        404 2017-02-07 22:46 /user/pmolnar/data/AdventureWorks/SalesTerritory.csv.gz
</pre>

In [ ]:
Territory_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=True)\
    .load('/user/pmolnar/data/AdventureWorks/SalesTerritory.csv.gz')

Orders_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=True)\
    .load('/user/pmolnar/data/AdventureWorks/ItemsOrdered.csv.gz')
Customers_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=True)\
    .load('/user/pmolnar/data/AdventureWorks/Customer.csv.gz')

In [ ]:
print Territory_df.count(), Orders_df.count(), Customers_df.count()

# Filtering (with)

### 1. Provide a list of employees that are married.

In [ ]:
Employees_df.where("MaritalStatus = 'M'").select(['EmployeeID', 'FirstName', 'LastName', 'MaritalStatus']).show()

### 2a. Show me a list of employees that have a lastname that begins with "R".

In [ ]:
Employees_df.select(['EmployeeID', 'FirstName', 'LastName']).where("LastName LIKE 'R%'").show()

In [ ]:
df = Employees[Employees.LastName.map(lambda x: x[0] == 'R')]
df[['EmployeeID', 'FirstName', 'LastName']]

### 2b. Show me a list of employees that have a lastname that ends with "r"

### 2c. Provide a list of employees that have a hyphenated lastname.

### 3a. Provide a list of employees that are on salary and have more than 35 vacation hours left.

### 3b. Show the same as above but limit it to American employees.

### 3c. Show the same as above but limit it to non-American employees.

### 4a. List the married employees with more than 35 vacation hours, only ones living in Washington state.

### 4b. Change the logic to include anyone who meets any of the 3 conditions (i.e., people who are either married, live in Washington state, or have more than 35 vacation hours left)

### 4c. Show the same as above, but only for Production Technicians

### 5a. List all employees living in Redmond, Seattle, and Bellevue, showing EmployeeID, FirstName, LastName, and City. Sort the list alphabetically by city.

### 5b. For the list above, make sure these are only in the state of Washington, just to be careful.

### 6. Provide a list of employees who have no title, whether it's a NULL or empty string.

### 7a. Provide a list of employees who have at least 60 vacation hours left.

### 7b. Provide a list of employees who have less than 60 vacation hours left.

### 7c. Show me employees who have more than 20 and less than 60 vacation hours left.

### 7d. If you did not use BETWEEN for 7c, do the same but use BETWEEN. If you did use BETWEEN for 7c, do it another way.

# Grouping

### 1a. What is the earliest birthdate for all employees?

In [ ]:
Employees_df.agg(F.min('BirthDate')).collect()

### 1b. Add to the above, the most recent birthdate for all employees

### 1c. Show the above results broken down by gender

### 1d. Show the above results broken down by gender, and salaried/hourly

### 2a. What are the average vacation hours for all employees?

### 2b. Add to the above, the minimum vacation hours for all employees

### 2c. Show the above results broken down and ordered by job title

In [ ]:
Employees_df.groupBy(['JobTitle']).agg(F.mean('VacationHours').alias('Average Vacation'), F.min('VacationHours').alias('Minimum Vacation')).show()

### 2d. Show the above results broken down by job title, and married/single employees

### 2e. Add to the above, the maximum vacation hours per group

### 2f. Show the above results broken down by job title, married/single employees, and State

### 2g. Show the above results but only for American employees

### 2h. Change the grouping above so it's broken down by married/single and State, no more job title

### 2i. Limit the results above to States where the average vacation hours is greater than 30

### 2j. Limit the results above to States where the average vacation hours is greater than 30 and the maximum vacation hours is less than 50

### 2k. Show the same results but only for non-American employees

### 3a. Report how many employees are in the company

In [ ]:
Employees_df.count()

### 3b. For the above report, show the number of employees per manager (hint: use ManagerID)

### 3c. Remove any manager ID's that are NULL from the results above

### 3d. Show the same results as above, but only for managers who have at least 5 employees

### 4a. List the average vacation hours of all employees

### 4b. Break down the results by State

### 4c. Break down the results by city and State

### 4d. Add something that shows the number of employees per city

### 4e. Sort the results by the city and state

### 4f. Make city and State a single column in the format of "City, State"

### 4g. Add a column that shows the difference between the maximum vacation hours and minimum vacation hours for each city

### 4h. Now sort the results by the new column created above

### 4i. Limit the results to cities that have more than 1 employee

### 4j. Limit the results to non-U.S. cities